In [23]:
from openai import OpenAI
import requests
from bs4 import BeautifulSoup
from IPython.display import display,Markdown
import gradio as gr

In [24]:
!ollama pull llama3.2:1b
MODEL="llama3.2:1b"
messages=[{"role":"system","content":"You are brochure generator assistant , visit all the text content in the website as well as the other other pages and create a funny and creative brochure in markdown"},
{"role":"user","content":""}]


pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest â � pulling manifest â ‹ pulling manifest â ™ pulling manifest â ¹ pulling manifest â ¸ pulling manifest â ¼ pulling manifest â ´ pulling manifest â ¦ pulling manifest â § pulling manifest â ‡ pulling manifest 
pulling 74701a8c35f6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.3 GB                         
pulling 966de95ca8a6... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 1.4 KB                         
pulling fcc5a6bec9da... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 7.7 KB                         
pulling a70ff7e570d9... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–� 6.0 KB                         
pulling 4f659a1e86d7... 100% â–•â–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–ˆâ–�  485 B                         
ver

In [25]:
#for scraping the website using beautifulsoup
headers = {
 "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36"
}
class WebSite:
    def __init__(self,url):
        response = requests.get(url,headers=headers)
        self.body = response.content
        soup = BeautifulSoup(self.body,"html.parser")
        self.title= soup.title.string if soup.title else "No Title Found"
        if soup.body:
            for irrelevant in soup.body(["script","img","style","input"]):
                irrelevant.decompose()
            self.text= soup.body.get_text(separator="\n",strip=True)
        else:
            self.text=""
        links= [link.get("href") for link in soup.find_all('a')]
        self.links=[link for link in links if link]

    
    

In [26]:

def generate_prompt(url):
    
    web = WebSite(url)
    user_prompt=f"The content of the website titled {web.title} is as follows \n"
    user_prompt+=web.text+"\n"
    user_prompt+="Other addition information can be found in the hyperlinks: \n"
    for link in web.links:
        user_prompt+=link+"\n"
    user_prompt+="Generate a funny and create brochure with above details"
    return user_prompt



In [27]:
def generate_brochure(url):
    prompt = generate_prompt(url)
    openai = OpenAI(base_url="http://localhost:11434/v1",api_key="ollama")
    for message in messages:
        if message["role"]=="user":
            message["content"]=prompt
    response = openai.chat.completions.create(model=MODEL,messages=messages)
    return response.choices[0].message.content


In [28]:
gr.Interface(fn=generate_brochure, inputs="textbox",outputs=[gr.Markdown(label="Response")],flagging_mode="never").launch(share=True)

* Running on local URL:  http://127.0.0.1:7872

Could not create share link. Please check your internet connection or our status page: https://status.gradio.app.
